In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy.stats import boxcox 
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
from scipy.linalg import pinv2, inv
import time
import numpy as np
import multiprocessing as mp
import pickle
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data(path):
    return pd.read_csv(path)
win10_df = load_data("../input/intrusion-detection-data/windows10_dataset.csv")

In [3]:
features = win10_df.drop(columns=['ts', 'type'], axis=1)
numeric_features = features.select_dtypes(exclude=['object'])
numeric_features.head(2)

,Processor_DPC_Rate,Process_Pool_Paged Bytes,Process_Working_Set_ Private,Process_Working_Set_Peak,Process_Page_File Bytes,Process_Virtual_Bytes Peak,Process_Page_File Bytes Peak,Process_Private_Bytes,Process_Elapsed_Time,Process_Virtual_Bytes,Process_Creating Process ID,Process_Pool Nonpaged Bytes,Process_Working Set,Process_ID Process,Process_Thread Count,Process_Priority Base,Process_Handle Count,Network_I(Intel R _82574L_GNC)TCP_APS,Network_I(Intel R _82574L_GNC) Packets Received Unknown,Network_I(Intel R _82574L_GNC) Packets Outbound Errors,Network_I(Intel R _82574L_GNC) Packets Received Discarded,Network_I(Intel R _82574L_GNC) Packets Outbound Discarded,Network_I(Intel R _82574L_GNC) Output Queue Length,Network_I(Intel R _82574L_GNC) Current Bandwidth,Network_I(Intel R _82574L_GNC) TCP Active RSC Connections,Network_I(Intel R _82574L_GNC) Offloaded Connections,Network_I(Intel R _82574L_GNC) Packets Received Errors,Memory Pool Paged Bytes,Memory Free & Zero Page List Bytes,Memory Cache Bytes Peak,Memory System Code Resident Bytes,Memory Available Bytes,Memory Commit Limit,Memory Available KBytes,Memory Cache Bytes,Memory Pool Nonpaged Bytes,Memory System Cache Resident Bytes,Memory Long-Term Average Standby Cache Lifetime (s),Memory Standby Cache Reserve Bytes,Memory System Code Total Bytes,Memory Standby Cache Core Bytes,Memory System Driver Resident Bytes,Memory Standby Cache Normal Priority Bytes,Memory Pool Paged Allocs,Memory Pool Nonpaged Allocs,Memory pct_ Committed Bytes In Use,Memory Free System Page Table Entries,Memory Available MBytes,Memory Modified Page List Bytes,Memory Committed Bytes,Memory System Driver Total Bytes,Memory Pool Paged Resident Bytes,LogicalDisk(_Total) pct_ Free Space,LogicalDisk(_Total) Current Disk Queue Length,LogicalDisk(_Total) Free Megabytes,label
0,4,15446344,1185267712,5639348224,2536640512,112000000000000,9375539200,2536640512,0,112000000000000,0,2447032,1319534592,0,789,0,29996,0,0,0,0,0,0,1000000000,0,0,0,284635136,4018176,815652864,1466368,261980160,6444666880,255840,18944000,89763840,0,48,5668864,3825664,157052928,2957312,95240192,164783,122023,53.693804,12229087,249,962560,3460386816,17117184,83619840,82.156219,0,167984,0
1,9,15448880,1188397056,5639348224,2535768064,112000000000000,9379713024,2535768064,0,112000000000000,0,2451920,1322864640,0,787,0,29992,0,0,0,0,0,0,1000000000,0,0,0,284782592,7233536,815652864,1466368,261197824,6444666880,255076,16445440,89747456,0,48,2158592,3825664,157036544,2985984,94769152,164891,122068,53.687639,12229106,249,1376256,3459989504,17158144,83984384,82.156219,0,167984,0


In [4]:
#scaling the data
def min_max_scaler(df):
    min_max_scaler_df = df.copy()
    for col in df.columns:
        min_max_scaler_df[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
    return min_max_scaler_df
min_max_df = min_max_scaler(numeric_features)
min_max_df.head(2)

,Processor_DPC_Rate,Process_Pool_Paged Bytes,Process_Working_Set_ Private,Process_Working_Set_Peak,Process_Page_File Bytes,Process_Virtual_Bytes Peak,Process_Page_File Bytes Peak,Process_Private_Bytes,Process_Elapsed_Time,Process_Virtual_Bytes,Process_Creating Process ID,Process_Pool Nonpaged Bytes,Process_Working Set,Process_ID Process,Process_Thread Count,Process_Priority Base,Process_Handle Count,Network_I(Intel R _82574L_GNC)TCP_APS,Network_I(Intel R _82574L_GNC) Packets Received Unknown,Network_I(Intel R _82574L_GNC) Packets Outbound Errors,Network_I(Intel R _82574L_GNC) Packets Received Discarded,Network_I(Intel R _82574L_GNC) Packets Outbound Discarded,Network_I(Intel R _82574L_GNC) Output Queue Length,Network_I(Intel R _82574L_GNC) Current Bandwidth,Network_I(Intel R _82574L_GNC) TCP Active RSC Connections,Network_I(Intel R _82574L_GNC) Offloaded Connections,Network_I(Intel R _82574L_GNC) Packets Received Errors,Memory Pool Paged Bytes,Memory Free & Zero Page List Bytes,Memory Cache Bytes Peak,Memory System Code Resident Bytes,Memory Available Bytes,Memory Commit Limit,Memory Available KBytes,Memory Cache Bytes,Memory Pool Nonpaged Bytes,Memory System Cache Resident Bytes,Memory Long-Term Average Standby Cache Lifetime (s),Memory Standby Cache Reserve Bytes,Memory System Code Total Bytes,Memory Standby Cache Core Bytes,Memory System Driver Resident Bytes,Memory Standby Cache Normal Priority Bytes,Memory Pool Paged Allocs,Memory Pool Nonpaged Allocs,Memory pct_ Committed Bytes In Use,Memory Free System Page Table Entries,Memory Available MBytes,Memory Modified Page List Bytes,Memory Committed Bytes,Memory System Driver Total Bytes,Memory Pool Paged Resident Bytes,LogicalDisk(_Total) pct_ Free Space,LogicalDisk(_Total) Current Disk Queue Length,LogicalDisk(_Total) Free Megabytes,label
0,0.020513,0.254279,0.347702,0.509926,0.274078,0.125998,0.676939,0.274078,NaN,0.125998,NaN,0.016574,0.142423,NaN,0.05419,NaN,0.227913,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.474041,0.001572,1.0,0.409611,0.089327,0.309638,0.089327,0.023806,0.335761,0.0,0.003126,0.005038,0.865616,0.607586,1.680291e-07,0.069145,0.258663,0.438832,0.386306,0.000017,0.089318,0.001114,0.191387,0.040099,0.180379,0.125954,0.0,0.121252,0.0
1,0.046154,0.254362,0.348839,0.509926,0.273947,0.125998,0.677290,0.273947,NaN,0.125998,NaN,0.016630,0.143232,NaN,0.05293,NaN,0.227830,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,0.474401,0.002831,1.0,0.409611,0.089055,0.309638,0.089055,0.020587,0.335599,0.0,0.003126,0.001918,0.865616,0.607523,1.696582e-07,0.068803,0.258832,0.438994,0.386221,0.000017,0.089318,0.001592,0.191353,0.042339,0.181282,0.125954,0.0,0.121252,0.0


In [5]:
numeric_features['target'] = win10_df['type'].values

In [6]:
le = LabelEncoder()
numeric_features['target'] = le.fit_transform(numeric_features['target'])

In [7]:
X = numeric_features.drop(columns = ['target'], axis=1)
y = numeric_features[['target']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train), np.array(y_test)

In [8]:
class ExtremeLearningMachine():
    def __init__(self, hidden_units, activation_function,  x, y, C, elm_type, one_hot=True, random_type=None):
        self.hidden_units = hidden_units
        self.activation_function = activation_function
        self.random_type = random_type
        self.x = x
        self.y = y
        self.C = C
        self.class_num = np.unique(self.y).shape[0]     
        self.beta = np.zeros((self.hidden_units, self.class_num))   
        self.elm_type = elm_type
        self.one_hot = one_hot

        # if classification problem and one_hot == True
        if elm_type == 'clf' and self.one_hot:
            self.one_hot_label = np.zeros((self.y.shape[0], self.class_num))
            for i in range(self.y.shape[0]):
                self.one_hot_label[i, int(self.y[i])] = 1

        # Randomly generate the weight matrix and bias vector from input to hidden layer
        # 'uniform': uniform distribution
        # 'normal': normal distribution
        if self.random_type == 'uniform':
            self.W = np.random.uniform(low=0, high=1, size=(self.hidden_units, self.x.shape[1]))
            self.b = np.random.uniform(low=0, high=1, size=(self.hidden_units, 1))
        if self.random_type == 'normal':
            self.W = np.random.normal(loc=0, scale=0.5, size=(self.hidden_units, self.x.shape[1]))
            self.b = np.random.normal(loc=0, scale=0.5, size=(self.hidden_units, 1))
        if self.random_type == "ES":
            self.W = jittered_weights
            self.b = np.random.uniform(low=0, high=1, size=(self.hidden_units, 1))

    # compute the output of hidden layer according to different activation function
    def __input2hidden(self, x):
        self.temH = np.dot(self.W, x.T) + self.b

        if self.activation_function == 'sigmoid':
            self.H = 1/(1 + np.exp(- self.temH))

        if self.activation_function == 'relu':
            self.H = self.temH * (self.temH > 0)

        if self.activation_function == 'sin':
            self.H = np.sin(self.temH)

        if self.activation_function == 'tanh':
            self.H = (np.exp(self.temH) - np.exp(-self.temH))/(np.exp(self.temH) + np.exp(-self.temH))

        if self.activation_function == 'leaky_relu':
            self.H = np.maximum(0, self.temH) + 0.1 * np.minimum(0, self.temH)

        return self.H

    # compute the output
    def __hidden2output(self, H):
        self.output = np.dot(H.T, self.beta)
        return self.output

    def fit(self, algorithm):
        self.time1 = time.clock()   # compute running time
        self.H = self.__input2hidden(self.x)
        if self.elm_type == 'clf':
            if self.one_hot:
                self.y_temp = self.one_hot_label
            else:
                self.y_temp = self.y
        if self.elm_type == 'reg':
            self.y_temp = self.y
        # no regularization
        if algorithm == 'no_re':
            self.beta = np.dot(pinv2(self.H.T), self.y_temp)
        # faster algorithm 1
        if algorithm == 'solution1':
            self.tmp1 = inv(np.eye(self.H.shape[0])/self.C + np.dot(self.H, self.H.T))
            self.tmp2 = np.dot(self.tmp1, self.H)
            self.beta = np.dot(self.tmp2, self.y_temp)
        # faster algorithm 2
        if algorithm == 'solution2':
            self.tmp1 = inv(np.eye(self.H.shape[0])/self.C + np.dot(self.H, self.H.T))
            self.tmp2 = np.dot(self.H.T, self.tmp1)
            self.beta = np.dot(self.tmp2.T, self.y_temp)
        self.time2 = time.clock()

        # compute the results
        self.result = self.__hidden2output(self.H)
        # If the problem if classification problem, the output is softmax
        if self.elm_type == 'clf':
            self.result = np.exp(self.result)/np.sum(np.exp(self.result), axis=1).reshape(-1, 1)

        # Evaluate training results
        # If problem is classification, compute the accuracy
        # If problem is regression, compute the RMSE
        if self.elm_type == 'clf':
            self.y_ = np.where(self.result == np.max(self.result, axis=1).reshape(-1, 1))[1]
            self.correct = 0
            for i in range(self.y.shape[0]):
                if self.y_[i] == self.y[i]:
                    self.correct += 1
            self.train_score = self.correct/self.y.shape[0]
        if self.elm_type == 'reg':
            self.train_score = np.sqrt(np.sum((self.result - self.y) * (self.result - self.y))/self.y.shape[0])
        train_time = str(self.time2 - self.time1)
        return self.beta, self.train_score, train_time
    def predict(self, x):
        self.H = self.__input2hidden(x)
        self.y_ = self.__hidden2output(self.H)
        if self.elm_type == 'clf':
            self.y_ = np.where(self.y_ == np.max(self.y_, axis=1).reshape(-1, 1))[1]

        return self.y_

    def score(self, x, y):
        self.prediction = self.predict(x)
        if self.elm_type == 'clf':
            self.correct = 0
            for i in range(y.shape[0]):
                if self.prediction[i] == y[i]:
                    self.correct += 1
            self.test_score = self.correct/y.shape[0]
        if self.elm_type == 'reg':
            self.test_score = np.sqrt(np.sum((self.result - self.y) * (self.result - self.y))/self.y.shape[0])

        return self.test_score
    def get_weights(self):
        return self.W

In [9]:
#Testing ELM model
sample_elm = ExtremeLearningMachine(hidden_units=32, activation_function="sigmoid", random_type="normal", 
                                   x = X_train, y=y_train, C=0.1, elm_type = "clf")
_, _, _ = sample_elm.fit("solution2")
sample_weights = sample_elm.get_weights()
sample_weights

array([[ 0.23328076,  0.67562548,  0.2277801 , ..., -0.59430342,
         0.26480276, -0.17888051],
       [ 0.23228828, -0.70497465,  0.21969228, ...,  0.61414529,
         0.29941716,  0.06240458],
       [-0.24449338, -0.57340437,  0.82750995, ...,  0.1146499 ,
        -0.21400138, -0.1808418 ],
       ...,
       [-0.21766716,  0.4899277 ,  0.33046986, ...,  0.55723329,
         0.36199316,  0.71630914],
       [-0.90260682, -0.33305723, -0.87161888, ..., -0.61458442,
        -0.60711783,  0.85682941],
       [ 0.02905643, -0.08147275, -0.81106267, ..., -0.56561787,
        -0.40234006, -0.40407585]])

In [10]:
#checking model accuracy
test_predictions = sample_elm.predict(X_test) 
print(accuracy_score(test_predictions, y_test))

0.6935050495691653


In [11]:
def fitness_func(sample_weights, y_train):
    model = ExtremeLearningMachine(hidden_units=32, activation_function="sigmoid", random_type="ES", 
                                  x=X_train, y=y_train, C = 0.1, elm_type="clf")
    _, _, _ = model.fit("solution1")
    train_predictions = model.predict(X_train)
    fit_score = f1_score(train_predictions, y_train, average="weighted")
    return fit_score

    

In [12]:
#testing fitness score function
jittered_weights = sample_weights + .0001
trial_score = fitness_func(jittered_weights, y_train)
trial_score

0.8168577335087389

In [13]:
#A quick function that jitters the primary parameters to diffrent state
def jitter(mother_params, state_dict):
    params_try = mother_params + SIGMA*state_dict # we will intiialize SIGMA later
    return params_try

In [14]:
def calculate_population_fitness(pop, mother_vector, y_train):
    fitness = np.zeros(pop.shape[0])
    for i, params in enumerate(pop):
        p_try = jitter(mother_vector, params)
        fitness[i] = fitness_func(p_try, y_train)
    return fitness

In [15]:
def test(mother_parameters, y_test):
    model = ExtremeLearningMachine(hidden_units=32, activation_function='sigmoid', random_type="ES", x=X_train, y=y_train, C=0.1, elm_type="clf") 
    _, _, _ = model.fit("solution1")
    predictions = model.predict(X_test)
    return accuracy_score(predictions, y_test)

In [16]:
SIGMA = 0.0001
LR = 0.001
POPULATION_SIZE=64
ITERATIONS = 500

In [17]:
num_params = sample_weights.shape[1]
num_params

56

In [18]:
np.random.seed(2022)
for iteration in tqdm(range(10)): 
    pop = np.random.randn(POPULATION_SIZE, num_params)
    fitness = calculate_population_fitness(pop, sample_weights, y_train)
    # normalize the fitness
    normalized_fitness = (fitness - np.mean(fitness)) / np.std(fitness)
    # update mother vector with the fitness values
    mother_vector = sample_weights + (LR / (POPULATION_SIZE * SIGMA)) * np.matmul(pop.T, normalized_fitness)
    if iteration % 1 == 0:
        reward = fitness_func(mother_vector, y_train)
        acc = test(mother_vector, y_test)
        print(f"Iteration: {iteration}, Reward:{reward}, Accuracy: {acc * 100}")

  0%|          | 0/10 [00:00<?, ?it/s]

Iteration: 0, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 1, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 2, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 3, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 4, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 5, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 6, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 7, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 8, Reward:0.8168577335087389, Accuracy: 69.35050495691652
Iteration: 9, Reward:0.8168577335087389, Accuracy: 69.35050495691652
